<a href="https://colab.research.google.com/github/code-cerebrum0/ai/blob/main/Tools_in_langchan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain-core langchain-community pydantic duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [3]:
pip install -U ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 49.6 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 5.4.0
    Uninstalling lxml-5.4.0:
      Successfully uninstalled lxml-5.4.0


In [4]:
from langchain_community.tools import DuckDuckGoSearchRun
search_tool= DuckDuckGoSearchRun()
results = search_tool.invoke('ipl news')

print(results)

Jun 5, 2025 — The official IPL website with live scores, match updates, team rankings, all the latest news, and videos . Follow your favorite teams and players! IPL 2023: Get the latest news on T20 IPL live score, points table, schedule... Get live cricket scores, cricket updates of upcoming International, domestic and T20 matches. Catch all the latest videos, news about cricket on ESPNcricinfo. To stay updated on the IPL 2025 schedule, fixtures, and all breaking news , follow TOI Sports on the website, YouTube channel, and across all major social media platforms. Get Live Cricket Scores, Scorecard, Schedules of International and Domestic cricket matches along with Latest News , Videos and ICC Cricket Rankings of Players on Cricbuzz.
